In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [2]:
plain_vocab = {}
EOS = '<eos>'
entity_vocab = {}
id2wb = {}

In [3]:
plain_lines = open('./input.txt').read().split('\n')

In [4]:
print(plain_lines)
for line in plain_lines:
    lt = line.split()
    for w in lt:
        if w not in plain_vocab:
            plain_vocab[w] = len(plain_vocab)

['香川真司 に 久々 の チャンス が 訪れる か と 思わ れ て い た が 、 物事 が そう 簡単 に 運ぶ わけ で は なかっ た 。', 'その 置か れ た 厳しい 現状 を 改めて 認識 さ せ られ た 。', 'ブンデスリーガ 第 20節 、 ダルムシュタット 対 ドルトムント 。', 'ドルトムント は 120分 の 激闘 の 末 、 PK戦 で ようやく 勝利 し た ドイツ杯 ヘルタ 戦 から 中2 日 で ダルムシュタット に 乗り込ん だ 。', 'ダツムシュタット は 現在 、 最下位 。', 'リーグ 前半 戦 の 対戦 で は 6 － 0 で 下し た 、 言っ て みれ ば " お客さん " だ 。', '週明け の 火曜日 に は チャンピオンズリーグ （ CL ） 決勝トーナメント 1回戦 ベンフィカ 戦 も 行なわ れる 。', '落とせ ない ノックアウト方式 の 2試合 に 挟ま れ た この ダルムシュタット 戦 は 、 普段 試合 に 出 て い ない 選手 にとって 大きな チャンス と 思わ れ て い た 。', '地元紙 で も 、 香川真司 を はじめ いつも の ベンチ メンバー が スタメン に 並ぶ と 予想 さ れ て い た 。', 'ドルトムント の 調子 そのもの は 芳しい と は 言え ない 。', '前節 ライプツィヒ 戦 、 ドイツ杯 ヘルタ 戦 と 、 難しい 試合 で 2連勝 こそ し た ものの 、 その 前 は マインツ に 引き分け て いる 。', '昨年末 は CL を 含め て 4戦 連続 引き分け という 有様 だっ た 。', 'トーマス・トゥヘル 監督 だけ で なく 、 クラブ 上層部 も 批判 に さらさ れる 日々 が 続く 。']


In [5]:
plain_vocab[EOS] = len(plain_vocab)-1
pv = len(plain_vocab)

In [6]:
entity_lines = open('./output.txt').read().split('\n')

In [7]:
for line in entity_lines:
    lt = line.split()
    for w in lt:
        if w not in entity_vocab:
            id = len(entity_vocab)
            entity_vocab[w] = id
            id2wb[id] = w

In [8]:
id = len(entity_vocab)
entity_vocab[EOS] = id-1
id2wb[id-1] = EOS
ev = len(entity_vocab)

In [9]:
def mk_ct(gh, ht):
    alp = []
    s = 0.0
    for i in range(len(gh)):
        s += np.exp(ht.dot(gh[i]))
    ct = np.zeros(100)
    for i in range(len(gh)):
        alpi = np.exp(ht.dot(gh[i]))/s
        ct += alpi * gh[i]
    ct = Variable(np.array([ct]).astype(np.float32))
    return ct

In [18]:
class ATT(chainer.Chain):
    def __init__(self, pv, ev, k):
        super(ATT, self).__init__(
            embedx = L.EmbedID(pv, k),
            embedy = L.EmbedID(ev, k),
            H = L.LSTM(k, k),
            Wc1 = L.Linear(k, k),
            Wc2 = L.Linear(k, k),
            W = L.Linear(k, ev),
        )
        
    def __call__(self, pline, eline):
        gh = []
        for i in range(len(pline)):
            wid = plain_vocab[pline[i]]
            x_k = self.embedx(Variable(np.array([wid], dtype=np.int32)))
            h = self.H(x_k)
            gh.append(np.copy(h.data[0]))
            
        x_k = self.embedx(Variable(np.array([plain_vocab[EOS]], dtype=np.int32)))
        tx = Variable(np.array([entity_vocab[eline[0]]], dtype=np.int32))
        h = self.H(x_k)
        ct = mk_ct(gh, h.data[0])
        h2 = F.tanh(self.Wc1(ct) + self.Wc2(h))
        accum_loss = F.softmax_cross_entropy(self.W(h2), tx)
        
        for i in range(len(eline)):
            wid = entity_vocab[eline[i]]
            x_k = self.embedy(Variable(np.array([wid], dtype=np.int32)))
            next_wid = entity_vocab[EOS] if ( i == len(eline) - 1) else entity_vocab[eline[i+1]]
            tx = Variable(np.array([next_wid], dtype=np.int32))
            h = self.H(x_k)
            ct = mk_ct(gh, h.data)
            h2 = F.tanh(self.Wc1(ct) + self.Wc2(h))
            loss = F.softmax_cross_entropy(self.W(h2), tx)
            accum_loss += loss
            
        return accum_loss
    
    def reset_state(self):
        self.H.reset_state()


In [19]:
demb = 100
model = ATT(pv, ev, demb)
optimizer = optimizers.Adam()
optimizer.setup(model)

for epoch in range(100):
    for i in range(len(plain_lines)-1):
        
        pln = plain_lines[i].split()
        plnr = pln[::-1]
        eln = entity_lines[i].split()
        model.reset_state()
        model.zerograds()
        loss = model(plnr, eln)
        loss.backward()
        loss.unchain_backward()
        optimizer.update()
        print(i, " finished")
    
    outfile = "attention-"+str(epoch)+".model"
    serializers.save_npz(outfile, model)

0  finished
1  finished
2  finished
3  finished
4  finished
5  finished
6  finished
7  finished
8  finished
9  finished
10  finished
11  finished
0  finished
1  finished
2  finished
3  finished
4  finished
5  finished
6  finished
7  finished
8  finished
9  finished
10  finished
11  finished
0  finished
1  finished
2  finished
3  finished
4  finished
5  finished
6  finished
7  finished
8  finished
9  finished
10  finished
11  finished
0  finished
1  finished
2  finished
3  finished
4  finished
5  finished
6  finished
7  finished
8  finished
9  finished
10  finished
11  finished
0  finished
1  finished
2  finished
3  finished
4  finished
5  finished
6  finished
7  finished
8  finished
9  finished
10  finished
11  finished
0  finished
1  finished
2  finished
3  finished
4  finished
5  finished
6  finished
7  finished
8  finished
9  finished
10  finished
11  finished
0  finished
1  finished
2  finished
3  finished
4  finished
5  finished
6  finished
7  finished
8  finished
9  finished
10  